In [1]:
import sys
import os
from pathlib import Path
sys.path.append(os.getcwd().replace("notebooks","utils"))

import pandas as pd
import numpy as np
from co2_functions import Clustering
import general_purpose as gp
import joblib
from sklearn.preprocessing import PowerTransformer,RobustScaler

Hey!, el módulo co2 ha sido importado correctamente 🥳
Módulo General Listo Para Usarse 💻


In [2]:
# df = gp.dataframes_charger("df_clusters_v1.csv")
# eliminar = ["use_intensity_pc","latitude","longitude","Year","Country",
#             "Energy_type","CODE_x","continent"]
# df = df.drop(columns=eliminar)

# path = "C:/Users/ferna/The_Bridge_CO2_ML_Project/src/data/processed/"
path2 = "C:/Users/ferna/OneDrive/Documentos/MIS_COSAS_DE_THE_BRIDGE/Proyectos_Machine_Learning/co2_ml/src/data/processed/"
# df.to_pickle(path + "vars_mod_final.pkl")
# df.to_pickle(path2 + "vars_mod_final.pkl")

In [3]:
df = pd.read_pickle(path2 + "vars_mod_final.pkl")
df.head()

,GDP,Population,Energy_production,Energy_consumption,CO2_emission,per_capita_production,Energy_intensity_by_GDP,balance,eficiency,energy_dependecy,co2_pc,clusters,energy_type
0,279.389,15182.60,0.150,0.155,0.0,104.341,5.670,-0.005,0.0,0.001,0.0,1,0
1,279.389,15182.60,5.000,0.812,0.0,104.341,5.670,4.188,0.0,0.003,0.0,3,3
2,279.389,15182.60,0.616,0.616,0.0,104.341,5.670,0.000,0.0,0.002,0.0,3,2
3,279.389,15182.60,0.001,0.002,0.0,104.341,5.670,-0.000,0.0,0.000,0.0,1,4
4,278.376,15597.88,0.156,0.161,0.0,110.257,6.178,-0.005,0.0,0.001,0.0,1,0


In [45]:
df = pd.DataFrame({
    "year":[1980,1995],
    "country":["Germany","France"],
    "gdp":[15,20],
    "pop":[5,10],
    "eprod":[30,50],
    "econs":[10,25],
    "co2e":[8,16],
    "percappro":[3.5,7.9],
    "eintengdp":[2.1,5.3],
    "balance":[0.3,1.5],
    "efi":[0.02,0.8],
    "edepen":[0.05,0.06],
    "co2pc":[0.001,0.002],
    "etype":[0,3]})
df

,year,country,gdp,pop,eprod,econs,co2e,percappro,eintengdp,balance,efi,edepen,co2pc,etype
0,1980,Germany,15,5,30,10,8,3.5,2.1,0.3,0.02,0.05,0.001,0
1,1995,France,20,10,50,25,16,7.9,5.3,1.5,0.80,0.06,0.002,3


In [52]:
class Final_Model:

    def __init__(self,year,country,df,gdp,pop,eprod,econs,co2e,percappro,
                    eintengdp,balance,efi,edepen,co2pc,etype):
        self.year = year
        self.country = country
        self.df = df
        self.gdp = gdp
        self.pop = pop
        self.eprod = eprod
        self.econs = econs
        self.co2e = co2e
        self.percappro = percappro
        self.eintengdp = eintengdp
        self.balance = balance
        self.efi = efi
        self.edepen = edepen
        self.co2pc = co2pc
        self.etype = etype
        self.df_preproc = Final_Model.preprocessing(self,PowerTransformer())
        self.ruta = Path(os.getcwd().replace("notebooks","model"))

    def registration(self):

        destino = Path(os.getcwd().replace("notebooks","data/processed"))

        df.loc[len(self.df)] = np.array([self.year,self.country,self.gdp,self.pop,
                                self.eprod,self.econs,self.co2e,self.percappro,
                                self.eintengdp,self.balance,self.efi,
                                self.edepen,self.co2pc,self.etype])

        new_df = self.df
        new_df.to_pickle(destino/"df_pruebas.pkl")

    def preprocessing(self,escalado):

        data_df = self.df
        nd_columns = [x for x in data_df.columns if x not in ["year","country"]]

        data_fit = data_df.loc[:,nd_columns]
        
        new_data = pd.DataFrame(
            data_df.loc[len(data_df)-1,nd_columns].values.reshape(1,-1),
            columns=nd_columns)

        scaler = escalado.fit(data_fit)
        
        return pd.DataFrame(scaler.transform(new_data),
                            columns=scaler.get_feature_names_out())


    def clustering(self):
        # 1. Preprocesado y selección de variables
        df = self.df_preproc
        cols_touse = [x for x in df.columns if x in ["co2e","eprod"]]
        clus_df = df.loc[len(df)-1,cols_touse]
        clus_df = pd.DataFrame(clus_df.values.reshape(1,-1),
                                columns=["CO2_emission","Energy_production"])

        # 2. carga del modelo
        clustering = joblib.load(self.ruta/"ClusteringModel.pkl")
        # 3. devuelve la predicción
        pred = clustering.predict(clus_df)
        return pred

    def regression(self):
        cluster = Final_Model.clustering(self)
        reg_vars = {
                        0:['balance', 'econs', 'eprod', 'co2e'],
                        1:['co2e', 'co2pc', 'percappro', 'econs'],
                        2:['gdp', 'pop', 'econs', 'co2e', 'balance'],
                        3:['co2e', 'eprod', 'balance', 'econs']
                    }

        if cluster == 0:
            df = self.df_preproc[reg_vars[0]]
            clus_df = df.loc[len(df)-1]
            clus_df = pd.DataFrame(clus_df.values.reshape(1,-1),
                                    columns=reg_vars[0])

            reg_0 = joblib.load(self.ruta/"reg_cluster0.pkl")
            pred = reg_0.predict(clus_df)
        elif cluster == 1:
            df = Final_Model.preprocessing(self,escalado=RobustScaler())
            df = self.df_preproc[reg_vars[1]]
            clus_df = df.loc[len(df)-1]
            clus_df = pd.DataFrame(clus_df.values.reshape(1,-1),
                                    columns=reg_vars[1])

            reg_1 = joblib.load(self.ruta/"reg_cluster1.pkl")
            pred = reg_1.predict(clus_df)
        elif cluster == 2:
            df = self.df_preproc[reg_vars[2]]
            clus_df = df.loc[len(df)-1]
            clus_df = pd.DataFrame(clus_df.values.reshape(1,-1),
                                    columns=reg_vars[2])

            reg_2 = joblib.load(self.ruta/"reg_cluster2.pkl")
            pred = reg_2.predict(clus_df)
        else:
            df = self.df_preproc[reg_vars[3]]
            clus_df = df.loc[len(df)-1]
            clus_df = pd.DataFrame(clus_df.values.reshape(1,-1),
                                    columns=reg_vars[3])
            reg_3 = joblib.load(self.ruta/"reg_cluster3.pkl")
            pred = reg_3.predict(clus_df)

        return pred

    def classification(self):
        # 1. selección de variables
        vars_rf = ['gdp', 'pop', 'econs', 'percappro','eintengdp', 'balance',
                    'edepen','co2pc','etype']
        clas_df = self.df_preproc[vars_rf]

        # 2. carga del modelo
        model_class = joblib.load(self.ruta/"RanFor_Classifier.pkl")
        # 3. devuelve la predicción
        pred = model_class.predict(clas_df)
        return pred

    def run_whole_model(self):
        
        Final_Model.registration(self)

        tag = Final_Model.classification(self)[0]
        efi = Final_Model.regression(self)[0]

        if tag == 0:
            print(f"""The efficiency predicted for your country is {efi}, what means
                    it is classified in the environmental group {tag}.This group
                    is characterized by the following description:
                    LOW PRODUCTION-HIGH CONTAMINATION
                    The energy production is low but it is not the lower compared
                    with the rest of the world energy producers. The production
                    is based on natural gas, petroleum and coal and because of
                    this energy mix the co2 emissions are high.
                    RECOMENDATION
                    Your efficiency can improve a lot since your energy production
                    mix is not optimal. Focus on changing your energy sources.
                    """)
        elif tag == 1:
            print(f"""The efficiency predicted for your country is {efi}, what means
                    it is classified in the environmental group {tag}.This group
                    is characterized by the following description:
                    LOW PRODUCTION-LOW CONTAMINATION
                    The energy production is low but the contamination it also is.
                    In this group the production comes mainly from petroleum but
                    also from renewables and natural gas. The energy mix is not ideal,
                    but the emitted co2 has no great impact on environment
                    RECOMENDATION
                    As the production remains steady the country can continues this
                    way. But if the aim is to increase energy production the mix
                    should be improved in order to lower the co2 emissions. Reinforce
                    renewables""")
        elif tag == 2:
            print(f"""The efficiency predicted for your country is {efi}, what means
                    it is classified in the environmental group {tag}.This group
                    is characterized by the following description:
                    VERY HIGH PRODUCTION-VERY HIGH CONTAMINATION
                    The energy production is very high and contamination too, so
                    you are one of the world's major suppliers. The production in
                    this group comes normally from petroleum, coal and natural
                    gas
                    RECOMENDATION
                    Your country has great impact on environmental care so it would
                    be good diversify the production mix enhancing renewables and
                    natural gas if possible. In any case, reducing coal and pretroleum
                    would be great.""")
        else:
            print(f"""The efficiency predicted for your country is {efi}, what means
                    it is classified in the environmental group {tag}.This group
                    is characterized by the following description:
                    GOOD BALANCE BETWEEN PRODUCTION AND CONTAMINATION
                    The production amount is good, coming from a good balanced production
                    mix and using all of them proportionally.
                    RECOMENDATION
                    Just keep this way, your country is environmental friendly and
                    knows how to balance production and world care.
                    """)



In [53]:
app = Final_Model(1930,"prueba",df,1000,5000,25000,1400,30,50,0.7,0.004,4,5,1,3)

In [60]:
app.run_whole_model()

The efficiency predicted for your country is 113.13630999999995, what means
                    it is classified in the environmental group 2.This group
                    is characterized by the following description:
                    VERY HIGH PRODUCTION-VERY HIGH CONTAMINATION
                    The energy production is very high and contamination too, so
                    you are one of the world's major suppliers. The production in
                    this group comes normally from petroleum, coal and natural
                    gas
                    RECOMENDATION
                    Your country has great impact on environmental care so it would
                    be good diversify the production mix enhancing renewables and
                    natural gas if possible. In any case, reducing coal and pretroleum
                    would be great.


c:\Users\ferna\miniconda3\envs\the_bridge_22\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- co2pc
- econs
- edepen
- eintengdp
- etype
- ...
Feature names seen at fit time, yet now missing:
- co2_pc
- energy_consumption
- energy_dependecy
- energy_intensity_by_gdp
- energy_type
- ...

  warnings.warn(message, FutureWarning)
c:\Users\ferna\miniconda3\envs\the_bridge_22\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- co2e
- econs
- pop
Feature names seen at fit time, yet now missing:
- co2_emission
- energy_consumption
- population

  warnings.warn(message, FutureWarning)


In [61]:
app.df

,year,country,gdp,pop,eprod,econs,co2e,percappro,eintengdp,balance,efi,edepen,co2pc,etype
0,1980,Germany,15,5,30,10,8,3.5,2.1,0.3,0.02,0.05,0.001,0
1,1995,France,20,10,50,25,16,7.9,5.3,1.5,0.8,0.06,0.002,3
2,1930,prueba,1000,5000,25000,1400,30,50,0.7,0.004,4,5,1,3
3,1930,prueba,1000,5000,25000,1400,30,50,0.7,0.004,4,5,1,3
4,1930,prueba,1000,5000,25000,1400,30,50,0.7,0.004,4,5,1,3
5,1930,prueba,1000,5000,25000,1400,30,50,0.7,0.004,4,5,1,3
6,1930,prueba,1000,5000,25000,1400,30,50,0.7,0.004,4,5,1,3
7,1930,prueba,1000,5000,25000,1400,30,50,0.7,0.004,4,5,1,3
8,1930,prueba,1000,5000,25000,1400,30,50,0.7,0.004,4,5,1,3
